In [2]:
from z3 import *
import math

## Utils

In [3]:
#TODO: sposta gli utils tutti qua su
def flatten(matrix):
    return [e for row in matrix for e in row]

#### Conversion from int to bin

In [4]:
def num_bits(x):
    return math.floor(math.log2(x)) + 1

def find_max_digits(s, l): # TODO Idea: optimize number of digits depending on cases
    return num_bits(max(sum(s), max(l)))

In [5]:
# TODO: usa BitVec come classe dei numeri binari e implementa tutte le funzioni su quelli. Possiamo anche usare BitVecVal per la conversione implicita
def int_to_bin(x, digits):  # TODO read: #https://ericpony.github.io/z3py-tutorial/guide-examples.htm
    x_bin = [(x%(2**(i+1)) // 2**i)==1 for i in range(digits-1,-1,-1)]
    return x_bin

def bin_to_int(x):
    n = len(x)
    x = sum(2**(n-i-1) for i in range(n) if x[i] == 1)
    return x

## Encodings

#### At most/least one & exactly one

In [6]:
def at_least_one_seq(bool_vars):
    return Or(bool_vars)

def at_most_one_seq(x, name):
    # bool_vars renamed to x for simplicity
    n = len(x)
    if n == 1:
        return True
    s = [Bool(f"s_{i}_{name}") for i in range(n-1)]     # N.B. len(s) = n-1 != n = len (x)
    
    clauses = []
    clauses.append(Or(Not(x[0]), s[0]))                 # x[0] -> s[0] (s[i] modeled as: s[i] is true if the SUM UP TO index i IS 1!) i.e. x_1 -> s_1 in math notation
    for i in range(1, n-1):
        clauses.append(Or(Not(x[i]), s[i]))             # these two clauses model (x[i] v s[i-1]) -> s[i]
        clauses.append(Or(Not(s[i-1]), s[i]))
        clauses.append(Or(Not(s[i-1]), Not(x[i])))      # this one models s[i-1] -> not x[i]
    clauses.append(Or(Not(s[-1]), Not(x[-1])))          # s[n-2] -> not x[n-1]  i.e. s_(n-1) -> s_n in the mathematical notation (1-based)
    return And(clauses)

def exactly_one_seq(bool_vars, name):
    return And(at_least_one_seq(bool_vars), at_most_one_seq(bool_vars, name))

#### Operations on binary numbers

In [7]:
def equal(v, u):
    assert(len(v) == len(u))
    return And([v[k]==u[k] for k in range(len(v))])

def all_false(v):
    return And([Not(v[k]) for k in range(len(v))])

def leq_same_digits(v, u, digits):
    """Encoding of v <= u, implementation with digits fixed

    Args:
        v (list[bool]): binary representation of v
        u (list[bool]): binary representation of u
        digits (int): number of digits of v and u

    Returns:
        Z3 formula: encoding of v <= u in binary considering their {digits} most significant bits
    """
    assert(len(v) == len(u) and len(u) == digits)
    if digits == 1:
        return Or(v[0]==u[0], And(Not(v[0]), u[0]))
    else:
        return Or(And(Not(v[0]), u[0]), 
                  And(v[0]==u[0], leq_same_digits(v[1:], u[1:], digits-1)))


def leq(v, u):
    """Encoding of v <= u, implementation with different digits btw v and u, just need digits(v) <= digits(u)

    Args:
        v (list[bool]): binary representation of v
        u (list[bool]): binary representation of u
        digits (int): number of digits of v and u

    Returns:
        Z3 formula: encoding of v <= u in binary considering their {digits} most significant bits
    """
    digits_v = len(v)
    digits_u = len(u)
    assert(digits_v <= digits_u)

    delta_digits = digits_u - digits_v
    if delta_digits == 0:   # i.e. digits_v == digits_u
        return leq_same_digits(v, u, digits_v)
    else:
        return Or(Or(u[:delta_digits]), leq_same_digits(v, u[delta_digits:], digits_v))




In [8]:
# # Testing of leq
# for i in range(1, 7):
#     for j in range(4, 16):
#         digits_i = num_bits(i)
#         i_bin = int_to_bin(i, digits_i)
#         digits_j = num_bits(j)
#         j_bin = int_to_bin(j, digits_j)

#         s = Solver()

#         s.add(leq(i_bin, j_bin))
#         if s.check() == z3.sat:
#             m = s.model()
#             print(f"{i} <= {j}")
#         else:
#             print(f"{i} > {j}")


In [9]:
def sum_bin_same_digits(a_bin, b_bin, d_bin, digits, name):
    """Encodes into a SAT formula the binary sum {a_bin + b_bin = d_bin}, each number having {digits} num of bits

    Args:
        a_bin (list[bool]): binary representation of a
        b_bin (list[bool]): binary representation of b
        d_bin (list[bool]): binary representation of d
        digits (int): number of bits of each number 
        name (str): string to identify carry boolean variables

    Returns:
        formula (Z3 expression): formula representing SAT encoding of binary sum
        c[0] (Bool): last carry of binary encoding
    """
    # c_k represents carry at bit position k
    c = [Bool(f"c_{k}_{name}") for k in range(digits+1)]
    c[-1] = False

    clauses = []
    for k in range(digits-1,-1,-1):
        # clauses.append((a_bin[k] == b_bin[k]) == (c[k+1] == d_bin[k]))    #TODO: check if it is faster
        clauses.append(d_bin[k] == Or(And(a_bin[k], b_bin[k], c[k+1]), And(a_bin[k], Not(b_bin[k]), Not(c[k+1])),
                                And(Not(a_bin[k]), b_bin[k], Not(c[k+1])), And(Not(a_bin[k]), Not(b_bin[k]), c[k+1])))
        clauses.append(c[k] == Or(And(a_bin[k],b_bin[k]), And(a_bin[k],c[k+1]), And(b_bin[k],c[k+1])))

    formula = And(clauses)
    return (formula, c[0])

def sum_bin(a_bin, b_bin, d_bin, name):
    """Encodes into a SAT formula the binary sum {a_bin + b_bin = d_bin}, with digits(a_bin) <= digits(b_bin) == digits(d_bin)

    Args:
        a_bin (list[bool]): binary representation of a
        b_bin (list[bool]): binary representation of b
        d_bin (list[bool]): binary representation of d
        name (str): string to identify carry boolean variables

    Returns:
        (Z3-expression): formula representing SAT encoding of binary sum
    """
    digits_a = len(a_bin)
    digits_b = len(b_bin)
    digits_d = len(d_bin)
    assert(digits_a <= digits_b and digits_b == digits_d)

    delta_digits = digits_b - digits_a

    if delta_digits == 0:
        formula, last_carry = sum_bin_same_digits(a_bin, b_bin, d_bin, digits_a, name)
        return And(formula, Not(last_carry))    # imposing no overflow

    sub_sum_formula, last_carry = sum_bin_same_digits(a_bin, b_bin[delta_digits:], d_bin[delta_digits:], digits_a, name)
    c = [Bool(f"c_propagated_{k}_{name}") for k in range(delta_digits)] + [last_carry]
    c[0] = False     # imposing no further overflow

    clauses = []
    for k in range(delta_digits-1, -1, -1):
        clauses.append(d_bin[k] == Xor(b_bin[k], c[k+1]))
        clauses.append(c[k] == And(b_bin[k], c[k+1]))

    return And(And(clauses), sub_sum_formula)

In [10]:
# ## Testing of sum_bin with variable num_digits
# for i in range(1, 10):
#     for j in range(100, 110):
#         digits_i = num_bits(i)
#         i_bin = int_to_bin(i, digits_i)
#         digits_j = num_bits(j) + 1
#         j_bin = int_to_bin(j, digits_j)
#         d_bin = [Bool(f"d_{k}") for k in range(digits_j)]

#         s = Solver()

#         s.add(sum_bin(i_bin, j_bin, d_bin, f"Prova_{i}_{j}"))
#         if s.check() == z3.sat:
#             m = s.model()
#             d = bin_to_int([1 if m.evaluate(d_bin[k]) else 0 for k in range(digits_j)])
#             print(f"{i} + {j} = {d}")
#         else:
#             print(f"UNSAT with {i} + {j}")


In [11]:
def conditional_sum_K_bin(x, alpha, delta, name):
    """Encodes into a SAT formula the constraint {delta = sum_over_j(alpha[j] | x[j] == True)}

    Args: 
        x (list[Bool]): list of Z3 Variables, i.e. x_j tells wether or not to add alpha_j to the sum
        alpha (list[list[bool]]): list of known coefficients, each one represented as list[bool] i.e. binary number, whose subset will be summed in the constraint
        delta (list[Bool]): list of Z3 Variables, which will be constrained to represent the sum
        name (string): to uniquely identify the created variables
    Returns:
        formula (Z3-expression): And of clauses representing SAT encoding of Linear Integer constraint
    
    """
    n = len(x)
    digits = len(delta)

    # matrix containing temporary results of sum_bin
    d = [[Bool(f"d_{j}_{k}_{name}") for k in range(digits)] for j in range(n-1)]    # j = 1..n-1 because last row will be delta
    d.append(delta)

    clauses = []

    # row 0
    diff_digits = digits - len(alpha[0])
    assert(diff_digits >= 0)
    clauses.append( And(Implies(x[0], And(all_false(d[0][:diff_digits]), equal(d[0][diff_digits:], alpha[0]))), # If x[0] == 1 then d_0 == alpha_0 (with eventual padding of zeros)
                        Implies(Not(x[0]), all_false(d[0]))))              # elif x[0] == 0 then d_0 == [0..0]

    # row j>1
    for j in range(1,n):
        clauses.append( And(Implies(x[j], sum_bin(alpha[j], d[j-1], d[j], f"{name}_{j-1}_{j}")),           # If c_j == 1 then d_j == d_j-1 + alpha_j 
                            Implies(Not(x[j]), equal(d[j], d[j-1]))))
        
    return And(clauses)

In [12]:
# # testing of conditional_sum_K_bin
# n = 4
# x = [Bool(f'x_{i}') for i in range(n)]
# alpha = [7, 9, 11, 4]
# digits = num_bits(max(e for e in alpha))
# alpha_bin = [int_to_bin(e, digits) for e in alpha]

# for comb in range(2**n):
#     s = Solver()
#     somma = 0
#     for i in range(n):
#         # print(comb, i, comb % (2**i))
#         if comb >= (2**i) and comb % (2**(i+1)) == 1:
#             print(comb, 2**i)
#             s.add(x[i])
#             somma += alpha[i]
#         else:
#             s.add(Not(x[i]))

#     delta = [Bool(f'delta_{j}') for j in range(8)]
#     s.add(conditional_sum_K_bin(x, alpha_bin, delta, 'ciao'))
#     s.check()
#     m = s.model()
#     print(f"{somma} ?= {bin_to_int([1 if m.evaluate(e) else 0 for e in delta])}")


# # print(s.check())
# # m = s.model()
# # print([m.evaluate(e) for e in x])
# # print([m.evaluate(e) for e in delta])

#### Linear Integer constraint over binary numbers

In [13]:
def LinearInteger_bin(x, alpha, beta, name):
    """Encodes into a SAT formula the linear constraint {sum_over_j(alpha[j] | x[j] == True) <= beta}

    Args: 
        x (list[Bool]): list of Z3 Variables, i.e. x_j tells wether or not to add alpha_j to the sum
        alpha (list[list[bool]]): list of known coefficients, each one represented as list[bool] i.e. binary number, whose subset will be summed in the constraint
        beta (list[bool]): binary representing the known term, i.e. the upper bound for the sum
        name (string): to uniquely identify the created variables
    Returns:
        formula (Z3-expression): And of clauses representing SAT encoding of Linear Integer constraint
    """
    n = len(x)
    digits = len(beta)

    clauses = []


    sum_result = [Bool(f"LIsum_result_{k}_{name}") for k in range(digits)]
    clauses.append(conditional_sum_K_bin(x, alpha, sum_result, name))
    
    clauses.append(leq(sum_result, beta)) 

    formula = And(clauses)
    return formula



In [14]:
# # Testing of LinearInteger_bin
# n = 4
# digits = 6
# x = [Bool(f'x_{i}') for i in range(n)]
# alpha = [7, 9, 11, 6]
# alpha_bin = [int_to_bin(e, digits) for e in alpha]
# beta = int_to_bin(5, digits)
# s = Solver()
# s.add(LinearInteger_bin(x, alpha_bin, beta, 'ciao'))
# s.add(Or(x))

# print(s.check())
# m = s.model()
# print([m.evaluate(e) for e in x])


#### Functions to display solution

In [15]:
def interest_variables(model, a):
    return [[j for j in range(len(a[0])) if model.evaluate(a[i][j])] for i in range(len(a))]

def displayMCP(result): # TODO: fai stampare anche le distanze percorse da ogni corriere, magari con il peso di ogni arco nel suo path
    # TODO: fai piu bello per UNSAT
    if type(result) == str and result == "UNSAT":
        print(result)
    else:
        orders, distances_bin, obj_value = result

        distances = [bin_to_int(d) for d in distances_bin]

        print(f"------Objective value: {obj_value}------")
        print(f"-------------Routes------------")
        m = len(orders)
        n = len(orders[0])
        for i in range(m):
            visited = []
            for t in range(n):
                found = False
                for j in range(n):
                    if orders[i][j][t]:
                        visited.append(j)
                        found = True
                if not found:
                    break

            print('Origin --> ' + ' --> '.join([str(vertex) for vertex in visited]) + f' --> Origin: travelled {distances[i]}')
            

#### Functions to display matrices of tensor and dist

In [16]:
### Add by Scaio

def evaluate(model, bools):
    if not isinstance(bools[0], list):
        return [1 if model.evaluate(b) else 0 for b in bools]
    return [evaluate(model, b) for b in bools]

# TODO: delete
def tensor(model, bools):
    m = len(bools)
    n = len(bools[0])
    return [[[model.evaluate(bools[i][j][k]) for k in range(n)] for j in range(n)] for i in range(m)]

def display_matrices(tensor, name):
    m = len(tensor)
    n = len(tensor[0])
    for i in range(m):
        print(f'{name}_{i} matrix:')
        for j in range(n):
            print(tensor[i][j])
        print()

def courier_dist(model, r_i, a_i, D, n): # Implemented with the assumption where r_i is the matrix nxn where 
                                         # j is the object and k indicates, with respect to the other columns, the order of delivery
    dist = 0
    actual_pos = n #the origin, it would be n+1
    next_pos = n
    for k in range(n): #sum(a_i)+1):
        for j in range(n):
            if model.evaluate(r_i[k][j]):
                next_pos = j
        dist += D[actual_pos][next_pos]
        actual_pos = next_pos
    dist += D[actual_pos][n] #the origin, it would be n+1
    return dist

def max_dist(model, r, a, D, m, n):
    return max([courier_dist(model, r[i], a[i], D, n) for i in range(m)])

###

#### Function to check that routes are all Hamiltonian cycles

In [17]:
# TODO: funzioni per noi, da rimuovere alla
def check_all_hamiltonian(tensor):
    m = len(tensor)
    for i in range(m):
        if not check_hamiltonian(tensor[i]):
            return False
    return True

def check_hamiltonian(matrix):
    n = len(matrix)
    visited = [0] * n
    v = n-1
    while visited[v] == 0:
        visited[v] = 1
        for k in range(n):
            if matrix[v][k] == True:
                v = k
                break
    num_vertices = sum(row.count(True) for row in matrix)
    return (sum(visited) == num_vertices)


#### Objective function section

In [42]:
def distances_def_constraint(distances, flat_r, flat_D_bin):
    """Defines the set of binary numbers {distances} as the sum of the binary numbers in {flat_D_bin} whose respective element in {flat_r} is True
        i.e. distances[i] = sum_over_j(flat_D_bin[j] | (flat_r[i])[j] == True)  forall i

    Args:
        distances (list[list[Bool]]): distances[i] will be constrained to be the sum of flat_D_bin[j] with respective flat_r[i][j] == True
        flat_r (list[list[Bool]]]): list of flattened matrices of Z3 Bool variables, each matrix flat_r[i] originally (i.e. before flattening) describing the route of courier i
        flat_D_bin (list[list[bool]]): flattened matrix of distances D, with each element converted to a binary as a list[bool]
    
    Returns:
        (Z3 formula): encoding of the definition of distances
    """
    m = len(distances)

    clauses = []

    for i in range(m):
        clauses.append(conditional_sum_K_bin(flat_r[i], flat_D_bin, distances[i], f"distances_def_{i}"))

    return And(clauses)    


def obj_function(model, distances):
    m = len(distances)
    maxdist = -1
    for i in range(m):
        dist = bin_to_int([1 if model.evaluate(distances[i][j]) else 0 for j in range(len(distances[i]))])
        maxdist = max(maxdist, dist)
    return maxdist

def AllLessEq_bin(distances, obj_value_bin, digits):
    m = len(distances)

    clauses = []

    for i in range(m):
        clauses.append(leq(distances[i], obj_value_bin)) #TODO: just a try without digits: , digits))

    return And(clauses)

## Assumptions

Assumpions that we can make:
1. $n >= m$
2. The distance matrix $D$ is quasimetric (more info [here](https://proofwiki.org/wiki/Definition:Quasimetric)) => Giving an item to courier who has none is always "same or better" than giving 2 to another courier and none to this one

(Note: 1. + 2. => can add implicit constraint "Every courier has at least one item assigned to it")

## Model 1

In [43]:
# TODO: add (and copy-implement) timeout decorator -> NO! implement it using time in the search cycle, and then return best model so far
def multiple_couriers_planning_1(m, n, l, s, D):
    """Model 1 in Z3 for the Multiple Couriers Planning problem

    Args:
        m (int): number of couriers
        n (int): number of items to deliver
        l (list[int]): l[i] represents the maximum load of courier i, for i = 1..m
        s (list[int]): s[j] represents the size of item j, for j = 1..n
        D (list[list[int]]): (n+1)x(n+1) matrix, with D[i][j] representing the distance from
                             distribution point i to distribution point j
                    
    """
    ## VARIABLES

    # a for assignments
    a = [[Bool(f"a_{i}_{j}") for j in range(n)] for i in range(m)]
    # a_ij = 1 indicates that courier i takes object j
    # O(m * n) vars     #TODO: add these complexity comments everywhere

    # r for routes
    r = [[[Bool(f"r_{i}_{j}_{k}") for k in range(n+1)] for j in range(n+1)] for i in range(m)]  
    # r_ijk = 1 indicates that courier i moves from delivery point j to delivery point k in his route
    # n+1 delivery points because considering Origin point as well, representes as n+1-th row and column, for each courier i
    # O(m * n^2) vars

    solver = Solver()

    ## CONSTRAINTS
    # Conversions:
    # TODO: can use the different num_bits optimization here as well now
    digits = find_max_digits(s, l)
    s_bin = [int_to_bin(s_j, digits) for s_j in s]
    l_bin = [int_to_bin(l_i, digits) for l_i in l]


    # Constraint 1: every object is assigned to one and only one courier
    for j in range(n):
        solver.add(exactly_one_seq([a[i][j] for i in range(m)], f"assignment_{j}"))
    
    # Constraint 2: every courier can't exceed its load capacity
    for i in range(m):
        solver.add(LinearInteger_bin(a[i], s_bin, l_bin[i], f"load_capacity_courier_{i}"))

    # Constraint 3: every courier has at least 1 item to deliver (implicit constraint, because n >= m and distance is quasimetric (from discussion forum))
    # TODO: check if it actually speeds up execution
    for i in range(m):
        solver.add(at_least_one_seq(a[i]))

    #TODO: better to use "clauses" list than doing so many solver.add below? Or call entirely a function maybe

    # Constraint 4: routes
    # last row and last column represent the Origin point
    t = [[[Bool(f"courier_{i}_delivers_{j}_as_{k}-th") for k in range(n)] for j in range(n)] for i in range(m)] #TODO 3: move down, just for debugging

    for i in range(m):
        # Constraint 4.1: diagonal is full of zeros, i.e. can't leave from j to go to j
        solver.add(And([Not(r[i][j][j]) for j in range(n+1)]))

        # Constraint 4.2: row j has a 1 iff courier i delivers object j
        # rows
        for j in range(n):
            solver.add(Implies(a[i][j], exactly_one_seq(r[i][j], f"courier_{i}_leaves_{j}")))  # If a_ij then exactly_one(r_ij)
            solver.add(Implies(Not(a[i][j]), all_false(r[i][j])))   # else all_false(r_ij)
        solver.add(exactly_one_seq(r[i][n], f"courier_{i}_leaves_origin"))    # exactly_one in origin point row === courier i leaves from origin

        # Constraint 4.3: column j has a 1 iff courier i delivers object j
        # columns
        for k in range(n):
            solver.add(Implies(a[i][k], exactly_one_seq([r[i][j][k] for j in range(n+1)], f"courier_{i}_reaches_{k}")))  # If a_ij then exactly_one(r_i,:,k)
            solver.add(Implies(Not(a[i][k]), all_false([r[i][j][k] for j in range(n+1)])))   # else all_false(r_i,:,k) 
            # TODO imply false just for the one that weren't alredy counted in the rows (Edo: "Cos?")
        solver.add(exactly_one_seq([r[i][j][n] for j in range(n+1)], f"courier_{i}_returns_to_origin"))         # exactly_one in origin point column === courier i returns to origin
     
        # Constraint 4.4: HAMILTONIAN CYCLE
        # Uses matrix t in order to check that the adjacency matrix r represents a Hamiltonian cycle
        # t_ijk = 1 iff object j is delivered as k-th element (i.e. at TIME=k, use of time as order) by courier i
        for j in range(n): 
            # assignments implications
            solver.add(Implies(a[i][j], exactly_one_seq(t[i][j], f'sometime_{j}_by_courier_{i}')))  # a_ij (j assigned to i) -> [t_ijk = True, for some k] (i delivers j at some time k)
            solver.add(Implies(Not(a[i][j]), all_false(t[i][j])))                                   # Not(a_ij) (j not assigned to i) -> [t_ijk = False, forall k] (i never delivers j)

            # can't deliver two items at the same time
            solver.add(at_most_one_seq([t[i][k][j] for k in range(n)], f"no_contemporary_delivery_{i}_{j}"))

            # 1st trip
            solver.add(Implies(r[i][n][j], t[i][j][0])) # r_inj (i rides n (origin) --> j) => t_ij0 (i delivers j as 1st item)

            # successive trips
            for h in range(1, n):
                for k in range(n):
                    solver.add(Implies(And(r[i][j][k], t[i][j][h-1]), t[i][k][h]))  # r_i,j,k AND t_i,j,h-1 (i goes from j to k AND he delivered j at time h-1) => t_i,k,h (k delivered at time h)
                
                solver.add(Implies(t[i][j][h], Or([And(r[i][k][j], t[i][k][h-1]) for k in range(n)])))       # if t_i,k,h (k delivered at time h) => [r_i,j,k AND t_i,j,h-1. for some j] (i came from some j to k AND he delivered that j at time h-1)
        ###
        

    # Optimization search

    # flatten r and D
    flat_r = [flatten(r[i]) for i in range(m)]
    flat_D = flatten(D)
    # convert flat_D to binary
    flat_D_bin = [int_to_bin(e, num_bits(e) if e > 0 else 1) for e in flat_D]

    # distances[i] := binary representation of the distance travelled by courier i
    solver.push()   # push the frame now in order to define distances with less digits every time  
    
    distance_upper_bound = sum([max(D[i]) for i in range(n+1)])     # TODO: maybe we can find a tighter upper bound on obj func
    dist_ub_digits = num_bits(distance_upper_bound) 
    distances = [[Bool(f"dist_bin_{i}_{k}") for k in range(dist_ub_digits)] for i in range(m)]
    # N.B. could either (1) find an upper bound to the objective value and use it immediately as a constraint (more information immediately, 
    # but is this allowed? can I use any algo to do so?)
    # or (2) run the model with no bound on the objective function the first time, then use the first value found as the initial bound
    # Question: can I do (1) and find a bound that is usually better than the bound found at (2)? If so, this would mean more info + better bound from the start!!
    
    # definition of distances using constraints
    solver.add(distances_def_constraint(distances, flat_r, flat_D_bin))
    
    model = None
    obj_value = None
    while solver.check() == z3.sat:
        model = solver.model()
        obj_value = obj_function(model, distances)
        print(f"This model obtained objective value: {obj_value}")

        if obj_value <= 1:
            break
        
        obj_value_digits = num_bits(obj_value - 1)  # <= obj_value - 1  <==>  < obj_value #TODO: could not do this and pass immediately the evaluation of distance with maximum value, smaller overhead...
                                                                                          # BUT then I couldn't print the line above!
        obj_value_bin = int_to_bin(obj_value - 1, obj_value_digits)

        Dists = evaluate(model, distances)  # save distances for final model display


        solver.pop()
        solver.push()
        # TODO: IMPORTANT - check if recreating the distances variables every time actually slows down the solver (probably in case of solver that uses explanations/heuristics!)
        # even tho they have less digits every time
        distances = [[Bool(f"dist_bin_{i}_{k}") for k in range(obj_value_digits)] for i in range(m)]        
        solver.add(distances_def_constraint(distances, flat_r, flat_D_bin)) # TODO: problemi: flat_D_bin[i] e distances non avranno gli stessi bits! distances molti di piu -> easy fix: implementa sum con generico numero di digits (Q: num(a) <= num(b) posso assumerlo??)
        solver.add(AllLessEq_bin(distances, obj_value_bin, obj_value_digits)) 

        # TODO: check why does it takes a lot of time between the two prints, maybe it's just the incremental solving or maybe is really inefficient the encoding
        print(f"obj_value_digits = {obj_value_digits}")
        print(f"dist_ub_digits = {dist_ub_digits}")
        
    
    # TODO: re-do thing above again but for BINARY_SEARCH, not LINEAR_SEARCH, then add parameter to the input of the model to choose between which one to use

    if model is None:
        return "UNSAT"
    
    # check that all couriers travel hamiltonian cycles
    R = evaluate(model, r)
    print(f"The tensor contains all hamiltonian cycles: {check_all_hamiltonian(R)}")

    T = evaluate(model, t)
    
    return (T, Dists, obj_value)


### Testing Model 1

In [44]:
m = 2
n = 2
l = [5, 5]
s = [1, 1]
D = [[0, 3, 3],
     [3, 0, 4],
     [3, 4, 0]]

displayMCP(multiple_couriers_planning_1(m, n, l, s, D))

This model obtained objective value: 8
obj_value_digits = 3
dist_ub_digits = 4
The tensor contains all hamiltonian cycles: True
------Objective value: 8------
-------------Routes------------
Origin --> 0 --> Origin: travelled 6
Origin --> 1 --> Origin: travelled 8


In [45]:
m = 1
n = 3
l = [15]
s = [3, 2, 4]
D = [[0, 3, 3, 2],
     [3, 0, 4, 3],
     [3, 4, 0, 3],
     [2, 3, 3, 0]]

displayMCP(multiple_couriers_planning_1(m, n, l, s, D))

This model obtained objective value: 12
obj_value_digits = 4
dist_ub_digits = 4
The tensor contains all hamiltonian cycles: True
------Objective value: 12------
-------------Routes------------
Origin --> 1 --> 0 --> 2 --> Origin: travelled 12


In [46]:
m = 3
n = 7
l = [15, 10, 7]
s = [1, 1, 1, 1, 1, 1, 1]
D = [[0, 3, 3, 6, 5, 6, 6, 2],
     [3, 0, 4, 3, 4, 7, 7, 3],
     [3, 4, 0, 7, 6, 3, 5, 3],
     [6, 3, 7, 0, 3, 6, 6, 4],
     [5, 4, 6, 3, 0, 3, 3, 3],
     [6, 7, 3, 6, 3, 0, 2, 4],
     [6, 7, 5, 6, 3, 2, 0, 4],
     [2, 3, 3, 4, 3, 4, 4, 0]]

displayMCP(multiple_couriers_planning_1(m, n, l, s, D))

This model obtained objective value: 18
obj_value_digits = 5
dist_ub_digits = 6
This model obtained objective value: 16
obj_value_digits = 4
dist_ub_digits = 6
This model obtained objective value: 14
obj_value_digits = 4
dist_ub_digits = 6
This model obtained objective value: 12
obj_value_digits = 4
dist_ub_digits = 6
The tensor contains all hamiltonian cycles: True
------Objective value: 12------
-------------Routes------------
Origin --> 0 --> 6 --> Origin: travelled 12
Origin --> 4 --> 5 --> 2 --> Origin: travelled 12
Origin --> 1 --> 3 --> Origin: travelled 10


In [49]:
m = 1
n = 3
l = [15]
s = [3, 2, 4]
D = [[0, 1, 1, 1],
     [1, 0, 1, 1],
     [1, 1, 0, 1],
     [1, 1, 1, 0]]

displayMCP(multiple_couriers_planning_1(m, n, l, s, D))

This model obtained objective value: 4
obj_value_digits = 2
dist_ub_digits = 3
The tensor contains all hamiltonian cycles: True
------Objective value: 4------
-------------Routes------------
Origin --> 0 --> 2 --> 1 --> Origin: travelled 4


In [50]:
# inst06.dat
m = 6
n = 8
l = [200, 100, 200, 300, 200, 100]
s = [16, 25, 21, 14, 19, 14, 6, 20]
D = [[0, 80, 131, 22, 41, 127, 87, 48, 113],
     [60, 0, 85, 82, 101, 81, 53, 106, 57],
     [141, 83, 0, 129, 182, 4, 64, 189, 28],
     [22, 82, 129, 0, 55, 125, 65, 66, 101],
     [41, 99, 172, 55, 0, 168, 118, 11, 154],
     [137, 81, 4, 125, 178, 0, 60, 185, 32],
     [77, 63, 64, 65, 118, 60, 0, 125, 36],
     [48, 126, 179, 60, 58, 175, 125, 0, 161],
     [113, 57, 28, 101, 154, 24, 36, 161, 0]]

displayMCP(multiple_couriers_planning_1(m, n, l, s, D))

This model obtained objective value: 322
obj_value_digits = 9
dist_ub_digits = 11
The tensor contains all hamiltonian cycles: True
------Objective value: 322------
-------------Routes------------
Origin --> 4 --> Origin: travelled 308
Origin --> 5 --> Origin: travelled 56
Origin --> 0 --> Origin: travelled 226
Origin --> 6 --> Origin: travelled 72
Origin --> 7 --> Origin: travelled 322
Origin --> 3 --> 1 --> 2 --> Origin: travelled 296


In [ ]:
m = 6
l = [190, 185, 185, 190, 195, 185]

n = 17
s = [11, 11, 23, 16, 2, 1, 24, 14, 20, 10, 11, 22, 2, 12, 9, 21, 10]

D =[[ 0, 20, 19, 28, 58, 48, 45, 32, 90, 61, 71, 59, 65, 46, 72, 51, 46, 66],
	[ 103, 0, 81, 107, 38, 110, 55, 94, 76, 123, 88, 76, 69, 86, 99, 113, 108, 106],
	[ 73, 41, 0, 80, 40, 29, 26, 13, 75, 42, 70, 58, 46, 27, 53, 32, 27, 47],
	[ 28, 48, 47, 0, 55, 59, 52, 60, 62, 58, 43, 31, 40, 21, 59, 48, 53, 61],
	[ 83, 38, 79, 87, 0, 102, 58, 92, 56, 103, 68, 56, 49, 67, 79, 94, 106, 86],
	[ 76, 44, 72, 83, 43, 0, 98, 16, 46, 64, 73, 61, 73, 67, 27, 40, 30, 43],
	[ 65, 55, 26, 64, 58, 55, 0, 39, 69, 68, 50, 38, 26, 31, 66, 58, 53, 56],
	[ 60, 28, 56, 67, 27, 16, 82, 0, 62, 48, 75, 63, 69, 51, 40, 24, 14, 34],
	[ 58, 76, 77, 81, 55, 46, 69, 62, 0, 56, 43, 31, 43, 53, 23, 80, 76, 30],
	[ 74, 79, 42, 84, 71, 35, 68, 51, 56, 0, 59, 47, 54, 37, 59, 24, 65, 43],
	[ 15, 35, 34, 43, 12, 43, 41, 47, 39, 47, 0, 12, 24, 10, 16, 37, 61, 32],
	[ 27, 47, 46, 50, 24, 55, 38, 59, 31, 47, 12, 0, 12, 22, 28, 49, 73, 30],
	[ 39, 59, 46, 38, 36, 56, 26, 59, 43, 54, 24, 12, 0, 19, 40, 45, 73, 37],
	[ 58, 68, 27, 57, 55, 38, 31, 40, 62, 37, 43, 31, 19, 0, 59, 27, 54, 50],
	[ 61, 68, 80, 84, 58, 27, 72, 40, 23, 59, 46, 34, 46, 56, 0, 64, 54, 16],
	[ 76, 55, 32, 83, 54, 11, 58, 27, 57, 24, 61, 49, 45, 27, 38, 0, 41, 54],
	[ 46, 14, 42, 53, 13, 21, 68, 14, 67, 34, 71, 59, 55, 37, 48, 10, 0, 47],
	[ 57, 61, 76, 75, 54, 43, 56, 34, 30, 43, 42, 30, 37, 50, 16, 57, 47, 0]]

displayMCP(multiple_couriers_planning_1(m, n, l, s, D))

This model obtained objective value: 477
D_digits = 7
obj_value_digits = 9
dist_ub_digits = 11
This model obtained objective value: 319
D_digits = 7
obj_value_digits = 9
dist_ub_digits = 11
This model obtained objective value: 318
D_digits = 7
obj_value_digits = 9
dist_ub_digits = 11
This model obtained objective value: 293
D_digits = 7
obj_value_digits = 9
dist_ub_digits = 11
This model obtained objective value: 248
D_digits = 7
obj_value_digits = 8
dist_ub_digits = 11
This model obtained objective value: 237
D_digits = 7
obj_value_digits = 8
dist_ub_digits = 11


KeyboardInterrupt: 